In [13]:
from zmax_datasets.datasets.usleep import USleepDataset
from pathlib import Path
import numpy as np

DATASETS_DIR = Path("/project/4180000.46/zmax_datasets/ppg_processed")
DATASET_NAME = "wearanize_plus"

dataset = USleepDataset(data_dir=DATASETS_DIR / DATASET_NAME)
print(dataset.n_recordings)
print(dataset.recording_ids)


97
['Sub001', 'Sub002', 'Sub004', 'Sub005', 'Sub007', 'Sub009', 'Sub010', 'Sub011', 'Sub012', 'Sub013', 'Sub014', 'Sub016', 'Sub017', 'Sub019', 'Sub020', 'Sub022', 'Sub023', 'Sub024', 'Sub025', 'Sub027', 'Sub028', 'Sub029', 'Sub031', 'Sub032', 'Sub033', 'Sub034', 'Sub035', 'Sub036', 'Sub038', 'Sub039', 'Sub040', 'Sub041', 'Sub043', 'Sub044', 'Sub045', 'Sub046', 'Sub047', 'Sub048', 'Sub050', 'Sub051', 'Sub053', 'Sub054', 'Sub056', 'Sub057', 'Sub059', 'Sub060', 'Sub061', 'Sub062', 'Sub063', 'Sub064', 'Sub065', 'Sub066', 'Sub067', 'Sub068', 'Sub071', 'Sub072', 'Sub073', 'Sub074', 'Sub075', 'Sub076', 'Sub079', 'Sub080', 'Sub081', 'Sub082', 'Sub084', 'Sub085', 'Sub086', 'Sub087', 'Sub088', 'Sub089', 'Sub090', 'Sub091', 'Sub092', 'Sub093', 'Sub094', 'Sub095', 'Sub096', 'Sub097', 'Sub099', 'Sub100', 'Sub101', 'Sub102', 'Sub103', 'Sub104', 'Sub105', 'Sub106', 'Sub107', 'Sub108', 'Sub109', 'Sub110', 'Sub111', 'Sub112', 'Sub118', 'Sub121', 'Sub124', 'Sub129', 'Sub130']


In [14]:
sample_recording = dataset.get_recording("Sub016")
print(sample_recording.data_types)


{'EMP_BVP_filtered': DataType(channel='EMP_BVP_filtered', sampling_rate=128.0), 'EMP_BVP_ibi': DataType(channel='EMP_BVP_ibi', sampling_rate=128.0), 'EMP_BVP_peaks': DataType(channel='EMP_BVP_peaks', sampling_rate=128.0), 'EMP_BVP_quality': DataType(channel='EMP_BVP_quality', sampling_rate=128.0), 'EMP_BVP_rate': DataType(channel='EMP_BVP_rate', sampling_rate=128.0), 'EMP_BVP_resampled': DataType(channel='EMP_BVP_resampled', sampling_rate=128.0), 'EMP_HR_resampled': DataType(channel='EMP_HR_resampled', sampling_rate=128.0), 'PPG_filtered': DataType(channel='PPG_filtered', sampling_rate=128.0), 'PPG_ibi': DataType(channel='PPG_ibi', sampling_rate=128.0), 'PPG_peaks': DataType(channel='PPG_peaks', sampling_rate=128.0), 'PPG_quality': DataType(channel='PPG_quality', sampling_rate=128.0), 'PPG_rate': DataType(channel='PPG_rate', sampling_rate=128.0), 'PPG_resampled': DataType(channel='PPG_resampled', sampling_rate=128.0)}


In [15]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from pathlib import Path
import pandas as pd

# Load all Empatica BVP signals
bvp_filtered = sample_recording.read_data_type("EMP_BVP_filtered")
bvp_peaks = sample_recording.read_data_type("EMP_BVP_peaks")
bvp_rate = sample_recording.read_data_type("EMP_BVP_rate")
bvp_quality = sample_recording.read_data_type("EMP_BVP_quality")
bvp_ibi = sample_recording.read_data_type("EMP_BVP_ibi")
emp_hr = sample_recording.read_data_type("EMP_HR_resampled")

# Create time axis in seconds for the signals
duration_seconds = len(bvp_rate.array.squeeze()) / bvp_rate.sample_rate
time = np.linspace(0, duration_seconds, len(bvp_rate.array.squeeze()))
print(f"Signal duration: {duration_seconds:.1f} seconds ({duration_seconds/60:.1f} minutes)")


2025-10-02 17:28:24.991 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: EMP_BVP_filtered
2025-10-02 17:28:25.540 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: EMP_BVP_peaks
2025-10-02 17:28:25.707 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: EMP_BVP_rate
2025-10-02 17:28:26.167 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: EMP_BVP_quality
2025-10-02 17:28:26.262 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: EMP_BVP_ibi
2025-10-02 17:28:26.529 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: EMP_HR_resampled


Signal duration: 22890.0 seconds (381.5 minutes)


In [16]:
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create widgets for controlling the window
start_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=duration_seconds - 60,  # Leave room for the window
    step=10,
    description='Start Time (s):',
    style={'description_width': 'initial'},
    layout={'width': '500px'}
)

window_size = widgets.Dropdown(
    options=[('10 seconds', 10), ('30 seconds', 30), ('1 minute', 60), ('5 minutes', 300)],
    value=60,
    description='Window Size:',
    style={'description_width': 'initial'}
)

# Create initial figure
fig = go.FigureWidget(make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.4, 0.2, 0.2, 0.2],
    subplot_titles=("Empatica BVP Signal and Peaks", "Signal Quality", "Heart Rate", "Inter-Beat Intervals")
))

# Initialize with empty traces
fig.add_trace(go.Scatter(name="BVP Filtered", line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(name="Peaks", mode='markers', marker=dict(color='red', size=8, symbol='circle')), row=1, col=1)
fig.add_trace(go.Scatter(name="Signal Quality", line=dict(color='purple'), fill='tozeroy'), row=2, col=1)
fig.add_trace(go.Scatter(name="BVP Heart Rate", line=dict(color='orange')), row=3, col=1)
fig.add_trace(go.Scatter(name="Empatica HR", line=dict(color='green')), row=3, col=1)
fig.add_trace(go.Scatter(name="Mean BVP HR", line=dict(color='red', dash='dash')), row=3, col=1)
# Add IBI trace and threshold lines
fig.add_trace(go.Scatter(name="IBI", line=dict(color='green')), row=4, col=1)
fig.add_trace(go.Scatter(name="Min Threshold", line=dict(color='red', dash='dash')), row=4, col=1)
fig.add_trace(go.Scatter(name="Max Threshold", line=dict(color='red', dash='dash')), row=4, col=1)

# Update layout
fig.update_layout(
    height=900,
    title=f"Empatica BVP Signal Analysis - Recording {str(sample_recording)}",
    showlegend=True,
    template="plotly_white"
)

# Update axes labels
fig.update_yaxes(title_text="Amplitude", row=1, col=1)
fig.update_yaxes(title_text="Quality Score", range=[0, 1], row=2, col=1)
fig.update_yaxes(title_text="Heart Rate (BPM)", range=[0, 200], row=3, col=1)
fig.update_yaxes(title_text="IBI (ms)", range=[0, 2500], row=4, col=1)  # Set range slightly above max threshold
fig.update_xaxes(title_text="Time (seconds)", row=4, col=1)

def update_plot(start_time, window_duration):
    # Calculate indices for the window
    start_idx = int(start_time * bvp_rate.sample_rate)
    end_idx = int((start_time + window_duration) * bvp_rate.sample_rate)
    
    # Create time array for the window
    time_window = np.linspace(start_time, start_time + window_duration, end_idx - start_idx)
    
    # Get signal segments
    filtered_signal = bvp_filtered.array.squeeze()[start_idx:end_idx]
    peaks_signal = bvp_peaks.array.squeeze()[start_idx:end_idx]
    quality_signal = bvp_quality.array.squeeze()[start_idx:end_idx]
    rate_signal = bvp_rate.array.squeeze()[start_idx:end_idx]
    ibi_signal = bvp_ibi.array.squeeze()[start_idx:end_idx]
    emp_hr_signal = emp_hr.array.squeeze()[start_idx:end_idx]
    
    # Update traces with new data
    with fig.batch_update():
        # Update BVP signal
        fig.data[0].x = time_window
        fig.data[0].y = filtered_signal
        
        # Update peaks
        peaks_idx = np.where(peaks_signal == 1)[0]
        if len(peaks_idx) > 0:
            fig.data[1].x = time_window[peaks_idx]
            fig.data[1].y = filtered_signal[peaks_idx]
        else:
            fig.data[1].x = []
            fig.data[1].y = []
        
        # Update template match
        fig.data[2].x = time_window
        fig.data[2].y = quality_signal
        
        # Update BVP heart rate
        fig.data[3].x = time_window
        fig.data[3].y = rate_signal
        
        # Update Empatica HR
        fig.data[4].x = time_window
        fig.data[4].y = emp_hr_signal
        
        # Update mean heart rate
        mean_hr = np.nanmean(rate_signal)
        fig.data[5].x = [time_window[0], time_window[-1]]
        fig.data[5].y = [mean_hr, mean_hr]
        fig.data[5].name = f"Mean BVP HR ({mean_hr:.1f} BPM)"
        
        # Update IBI plot
        # Only plot non-zero IBI values
        valid_ibi_mask = ibi_signal > 0
        valid_times = time_window[valid_ibi_mask]
        valid_ibis = ibi_signal[valid_ibi_mask]
        
        fig.data[6].x = valid_times
        fig.data[6].y = valid_ibis
        
        # Update threshold lines
        fig.data[7].x = [time_window[0], time_window[-1]]  # Min threshold
        fig.data[7].y = [300, 300]  # 300ms threshold
        
        fig.data[8].x = [time_window[0], time_window[-1]]  # Max threshold
        fig.data[8].y = [2000, 2000]  # 2000ms threshold

# Create the interactive plot
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_plot(start_slider.value, window_size.value)

# Link the widgets to the update function
start_slider.observe(on_change)
window_size.observe(on_change)

# Display widgets and initial plot
display(widgets.HBox([start_slider, window_size]))
display(fig)

# Initialize the plot
update_plot(start_slider.value, window_size.value)


FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'name': 'BVP Filtered',
              'type': 'scatter',
              'uid': 'f77349fc-905f-4d34-adff-3e528c39d21b',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'marker': {'color': 'red', 'size': 8, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Peaks',
              'type': 'scatter',
              'uid': '06dc08e9-bc85-4408-bce8-f6557af348d7',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'fill': 'tozeroy',
              'line': {'color': 'purple'},
              'name': 'Signal Quality',
              'type': 'scatter',
              'uid': '5edbfc51-1bb0-468f-b7f6-4e73132eb631',
              'xaxis': 'x2',
              'yaxis': 'y2'},
             {'line': {'color': 'orange'},
              'name': 'BVP Heart Rate',
              'type': 'scatter',
              'uid': 'bbb72fa5-886f-479d-8467-5b5c3308c84b',
              'xaxis'